In [ ]:
# install maerials project api
# requirements 
 # - python3.10
 # - ase

#!pip install mp-api

In [ ]:
MP_key = "Fs12lUqj7D3cUaQZWPhJvmXtQHSFeOp9"

## perform a test 

In [ ]:
from mp_api.client import MPRester
with MPRester(MP_key) as mpr:
    docs = mpr.materials.summary.search(
        band_gap=(3, None), is_stable=True, fields=["material_id"]
    )
    stable_mpids = [doc.material_id for doc in docs]
    ## -- Alternative directly using energy above hull:
    docs = mpr.materials.summary.search(
        band_gap=(1, None), energy_above_hull=(0, 0), fields=["material_id"]
    )
    stable_mpids = [doc.material_id for doc in docs]

# API to read the MPtrj database

In [ ]:
from mp_api.client import MPRester
from ase.io import read
from ase.visualize import view
from collections import Counter
import matplotlib.pyplot as plt
import os

with MPRester(MP_key) as mpr:
    docs = mpr.materials.summary.search(
        spacegroup_number=1, is_metal=False, theoretical=False, num_elements=(1,None), is_stable=True, fields=["material_id"]
    )
    stable_mpids = [doc.material_id for doc in docs]


# Selecting non-metallic crystals in spacegroup 1

In [ ]:
# List to store all structures
spacegroup_1 = []

# Temporary file for storing CIF data
temp_file = "temp.cif"

# Dictionary to keep track of chemical species
chemical_species_counts = Counter()

# Loop over all stable materials
for mpid in stable_mpids:
    # Delete temporary file if it exists
    if os.path.isfile(temp_file):
        os.remove(temp_file)
    
    # Retrieve and save the structure to CIF
    structure = mpr.get_structure_by_material_id(mpid)
    structure.to(filename=temp_file)
    
    # Read structure as ASE Atoms object and add to list
    atoms = read(temp_file)
    spacegroup_1.append(atoms)
    
    # Update chemical species count
    chemical_species_counts.update(atoms.get_chemical_symbols())

# Visualize all collected structures
view(spacegroup_1)

# Plot histogram of chemical species with a legend
plt.figure(figsize=(10, 6))
plt.bar(chemical_species_counts.keys(), chemical_species_counts.values(), color='blue', label="Space Group = 1\nMetals=False\nNum_Species=2-4")
plt.xlabel("Chemical Elements")
plt.ylabel("Count")
plt.title("Histogram of Chemical Species")
plt.xticks(rotation=45)
plt.legend(loc="upper right")  # Display the legend in the upper-right corner
plt.tight_layout()
plt.show()

# 230 single-element crystals

## test-1

In [ ]:
from mp_api.client import MPRester
from ase.io import read
from ase.visualize import view
from collections import Counter
import matplotlib.pyplot as plt
import os

num_spacegroup = 230

with MPRester(MP_key) as mpr:
    for sg in range(1,num_spacegroup):
        print(sg)
        docs = mpr.materials.summary.search(
            spacegroup_number=sg, is_metal=False, theoretical=False, num_elements=(1,1), is_stable=True, fields=["material_id"]
        )
        stable_mpids = [doc.material_id for doc in docs]
    
        # List to store all structures
        spacegroup_1 = []
        
        # Temporary file for storing CIF data
        temp_file = "temp.cif"
        
        # Dictionary to keep track of chemical species
        chemical_species_counts = Counter()
        
        # Loop over all stable materials
        for mpid in stable_mpids:
            # Delete temporary file if it exists
            if os.path.isfile(temp_file):
                os.remove(temp_file)
            
            # Retrieve and save the structure to CIF
            structure = mpr.get_structure_by_material_id(mpid)
            structure.to(filename=temp_file)
            
            # Read structure as ASE Atoms object and add to list
            atoms = read(temp_file)
            spacegroup_1.append(atoms)
            
            # Update chemical species count
            chemical_species_counts.update(atoms.get_chemical_symbols())
        
        # Visualize all collected structures
        view(spacegroup_1)
        
        # Plot histogram of chemical species with a legend
        plt.figure(figsize=(10, 6))
        plt.bar(chemical_species_counts.keys(), chemical_species_counts.values(), color='blue', label="Space Group = 1\nMetals=False\nNum_Species=2-4")
        plt.xlabel("Chemical Elements")
        plt.ylabel("Count")
        plt.title("Histogram of Chemical Species")
        plt.xticks(rotation=45)
        plt.legend(loc="upper right")  # Display the legend in the upper-right corner
        plt.tight_layout()
        plt.show()

# test-2

In [ ]:
from mp_api.client import MPRester
from ase.io import read
from ase.visualize import view
from collections import Counter
import matplotlib.pyplot as plt
import os

MP_key = "Fs12lUqj7D3cUaQZWPhJvmXtQHSFeOp9"

num_spacegroup = 230
temp_file = "temp.cif"

single_non_metallice_structures = []

with MPRester(MP_key) as mpr:
    for sg in range(1, num_spacegroup + 1):
        # Skip if space group name is not defined (incomplete dictionary)
        # if sg not in spacegroup_names:
        #     continue
        
        # Retrieve materials with the current space group
        docs = mpr.materials.summary.search(
            spacegroup_number=sg, is_metal=False, theoretical=False, num_elements=(1, 1), is_stable=True, fields=["material_id"]
        )
        stable_mpids = [doc.material_id for doc in docs]
        # List to store all structures for current space group
        spacegroup_structures = []
        
        # Dictionary to keep track of chemical species counts
        chemical_species_counts = Counter()
        
        for mpid in stable_mpids:
            # Ensure the temporary file does not already exist
            if os.path.isfile(temp_file):
                os.remove(temp_file)
            
            # Retrieve structure and save to CIF
            structure = mpr.get_structure_by_material_id(mpid)
            structure.to(filename=temp_file)
            
            # Read structure as ASE Atoms object
            atoms = read(temp_file)
            
            # Only add structure if atoms are present
            if len(atoms) > 0:
                spacegroup_structures.append(atoms)
                chemical_species_counts.update(atoms.get_chemical_symbols())
        
        # Proceed with viewing and plotting only if there are structures
        if spacegroup_structures:
            single_non_metallice_structures.append(spacegroup_structures)
            # View the collected structures
            #view(spacegroup_structures)
            
            # Plot histogram for chemical species
            plt.figure(figsize=(10, 6))
            plt.bar(chemical_species_counts.keys(), chemical_species_counts.values(), color='blue')
            plt.xlabel("Chemical Elements")
            plt.ylabel("Count")
            plt.title(f"Histogram of Chemical Species for Space Group {sg}")
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
        
        # Remove temp file if it still exists
        if os.path.isfile(temp_file):
            os.remove(temp_file)


In [ ]:
single_non_metallice_structures

In [ ]:
from mp_api.client import MPRester
from ase.io import read
from ase.visualize import view
from collections import Counter
import matplotlib.pyplot as plt
import os

MP_key = "Fs12lUqj7D3cUaQZWPhJvmXtQHSFeOp9"

num_spacegroup = 230
temp_file = "temp.cif"

single_non_metallice_structures = []

with MPRester(MP_key) as mpr:
    for sg in range(1, num_spacegroup + 1):
        # Skip if space group name is not defined (incomplete dictionary)
        # if sg not in spacegroup_names:
        #     continue
        
        # Retrieve materials with the current space group
        docs = mpr.materials.summary.search(
            spacegroup_number=sg, is_metal=False, num_elements=(1, 1), fields=["material_id"]
        )
        stable_mpids = [doc.material_id for doc in docs]
        
        # List to store all structures for current space group
        spacegroup_structures = []
        
        # Dictionary to keep track of chemical species counts
        chemical_species_counts = Counter()
        
        for mpid in stable_mpids:
            # Ensure the temporary file does not already exist
            if os.path.isfile(temp_file):
                os.remove(temp_file)
            
            # Retrieve structure and save to CIF
            structure = mpr.get_structure_by_material_id(mpid)
            structure.to(filename=temp_file)
            
            # Read structure as ASE Atoms object
            atoms = read(temp_file)
            
            # Only add structure if atoms are present
            if len(atoms) > 0:
                spacegroup_structures.append(atoms)
                chemical_species_counts.update(atoms.get_chemical_symbols())
        
        # Proceed with viewing and plotting only if there are structures
        if spacegroup_structures:
            single_non_metallice_structures.append(spacegroup_structures)
            # View the collected structures
            #view(spacegroup_structures)
            
            # Plot histogram for chemical species
            plt.figure(figsize=(10, 6))
            plt.bar(chemical_species_counts.keys(), chemical_species_counts.values(), color='blue')
            plt.xlabel("Chemical Elements")
            plt.ylabel("Count")
            plt.title(f"Histogram of Chemical Species for Space Group {sg}")
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
        
        # Remove temp file if it still exists
        if os.path.isfile(temp_file):
            os.remove(temp_file)


# final code to count single element non-metallic structure as per space groups

In [ ]:
from mp_api.client import MPRester
from ase.io import read
from collections import Counter
import matplotlib.pyplot as plt
import os

MP_key = "Fs12lUqj7D3cUaQZWPhJvmXtQHSFeOp9"

# Full dictionary of space group names (only showing a partial example for brevity)
spacegroup_names = {
    1: "P1", 2: "P-1", 
    3: "P2", 4: "P21", 5: "C2", 6: "Pm", 7: "Pc", 8: "Cm", 9: "Cc", 
    10: "P2/m", 11: "P21/m", 12: "C2/m", 13: "P2/c", 14: "P21/c", 15: "C2/c", 
    16: "P222", 17: "P2221", 18: "P21212", 19: "P212121", 20: "C2221", 21: "C222", 22: "F222", 23: "I222", 24: "I212121",
    25: "Pmm2", 26: "Pmc21", 27: "Pcc2", 28: "Pma2", 29: "Pca21", 30: "Pnc2", 31: "Pmn21", 32: "Pba2", 33: "Pna21", 
    34: "Pnn2", 35: "Cmm2", 36: "Cmc21", 37: "Ccc2", 38: "Amm2", 39: "Aem2", 40: "Ama2", 41: "Aea2", 
    42: "Fmm2", 43: "Fdd2", 44: "Imm2", 45: "Iba2", 46: "Ima2", 
    47: "Pmmm", 48: "Pnnn", 49: "Pccm", 50: "Pban", 51: "Pmma", 52: "Pnna", 53: "Pmna", 54: "Pcca", 55: "Pbam", 
    56: "Pccn", 57: "Pbcm", 58: "Pnnm", 59: "Pmmn", 60: "Pbcn", 61: "Pbca", 62: "Pnma", 
    63: "Cmcm", 64: "Cmce", 65: "Cmmm", 66: "Cccm", 67: "Cmme", 68: "Cmme", 69: "Ccca", 
    70: "Fmmm", 71: "Fddd", 72: "Immm", 73: "Ibam", 74: "Ibca", 75: "Imma", 
    76: "P4", 77: "P41", 78: "P42", 79: "P43", 80: "I4", 81: "I41", 
    82: "P-4", 83: "I-4", 
    84: "P4/m", 85: "P42/m", 86: "P4/n", 87: "P42/n", 88: "I4/m", 89: "I41/a", 
    90: "P422", 91: "P4212", 92: "P4122", 93: "P41212", 94: "P4222", 95: "P42212", 96: "P4322", 97: "P43212", 
    98: "I422", 99: "I4122", 
    100: "P4mm", 101: "P4bm", 102: "P42cm", 103: "P42nm", 104: "P4cc", 105: "P4nc", 106: "P42mc", 107: "P42bc", 
    108: "I4mm", 109: "I4cm", 110: "I41md", 111: "I41cd", 
    112: "P-42m", 113: "P-42c", 114: "P-421m", 115: "P-421c", 116: "P-4m2", 117: "P-4c2", 118: "P-4b2", 119: "P-4n2", 
    120: "I-4m2", 121: "I-4c2", 122: "I-42m", 123: "I-42d", 
    124: "P4/mmm", 125: "P4/mcc", 126: "P4/nbm", 127: "P4/nnc", 128: "P4/mbm", 129: "P4/mnc", 130: "P4/nmm", 
    131: "P4/ncc", 132: "P42/mmc", 133: "P42/mcm", 134: "P42/nbc", 135: "P42/nnm", 136: "P42/mbc", 137: "P42/mnm", 
    138: "P42/nmc", 139: "P42/ncm", 140: "I4/mmm", 141: "I4/mcm", 142: "I41/amd", 143: "I41/acd", 
    144: "P3", 145: "P31", 146: "P32", 147: "R3", 
    148: "P-3", 149: "R-3", 
    150: "P312", 151: "P321", 152: "P3112", 153: "P3121", 154: "P3212", 155: "P3221", 156: "R32", 
    157: "P3m1", 158: "P31m", 159: "P3c1", 160: "P31c", 161: "R3m", 162: "R3c", 
    163: "P-31m", 164: "P-31c", 165: "P-3m1", 166: "P-3c1", 167: "R-3m", 168: "R-3c", 
    169: "P6", 170: "P61", 171: "P65", 172: "P62", 173: "P64", 174: "P63", 
    175: "P-6", 176: "P6/m", 177: "P63/m", 
    178: "P622", 179: "P6122", 180: "P6522", 181: "P6222", 182: "P6422", 183: "P6322", 
    184: "P6mm", 185: "P6cc", 186: "P63cm", 187: "P63mc", 
    188: "P-6m2", 189: "P-6c2", 190: "P-62m", 191: "P-62c", 
    192: "P6/mmm", 193: "P6/mcc", 194: "P63/mcm", 
    195: "P23", 196: "F23", 197: "I23", 198: "P213", 199: "I213", 
    200: "Pm-3", 201: "Pn-3", 202: "Fm-3", 203: "Fd-3", 204: "Im-3", 
    205: "Pa-3", 206: "Ia-3", 
    207: "P432", 208: "P4232", 209: "F432", 210: "F4132", 211: "I432", 212: "P4332", 213: "P4132", 
    214: "I4132", 215: "P-43m", 216: "F-43m", 217: "I-43m", 218: "P-43n", 219: "F-43c", 220: "I-43d", 
    221: "Pm-3m", 222: "Pn-3n", 223: "Pm-3n", 224: "Pn-3m", 225: "Fm-3m", 226: "Fm-3c", 227: "Fd-3m", 
    228: "Fd-3c", 229: "Im-3m", 230: "Ia-3d"
}


num_spacegroup = 230
temp_file = "temp.cif"

# Dictionary to store counts for each element across all space groups
all_chemical_species_counts = {}

with MPRester(MP_key) as mpr:
    for sg in range(1, num_spacegroup + 1):
        # Skip if space group name is not defined (incomplete dictionary)
        if sg not in spacegroup_names:
            continue
        
        # Retrieve materials with the current space group
        docs = mpr.materials.summary.search(
            spacegroup_number=sg, is_metal=False, num_elements=(1, 1), fields=["material_id"]
        )
        stable_mpids = [doc.material_id for doc in docs]
        
        # List to store structures for this space group
        spacegroup_structures = []
        
        # Dictionary to keep track of chemical species counts
        chemical_species_counts = Counter()
        
        for mpid in stable_mpids:
            # Ensure the temporary file does not already exist
            if os.path.isfile(temp_file):
                os.remove(temp_file)
            
            # Retrieve structure and save to CIF
            structure = mpr.get_structure_by_material_id(mpid)
            structure.to(filename=temp_file)
            
            # Read structure as ASE Atoms object
            atoms = read(temp_file)
            
            # Only add structure if atoms are present
            if len(atoms) > 0:
                spacegroup_structures.append(atoms)
                chemical_species_counts.update(atoms.get_chemical_symbols())
        
        # Store chemical species counts for this space group if there are structures
        if spacegroup_structures:
            all_chemical_species_counts[spacegroup_names[sg]] = chemical_species_counts
        
        # Remove temp file if it still exists
        if os.path.isfile(temp_file):
            os.remove(temp_file)

# Plot all space groups in a single figure
plt.figure(figsize=(10, 5))

# Plot data for each space group
for spacegroup, species_counts in all_chemical_species_counts.items():
    plt.bar(species_counts.keys(), species_counts.values(), label=spacegroup)

# Label axes and set title
plt.xlabel("Chemical Elements")
plt.ylabel("Count")
plt.title("Histogram of Chemical Species across Space Groups")
plt.xticks(rotation=45)

# Set up the legend inside the plot area
plt.legend(
    loc="upper right", 
    fontsize=9, 
    title="Space Groups",
    ncol=5,  # Adjust number of columns for better readability
    columnspacing=1.0,  # Adjust spacing between columns
    handletextpad=0.5  # Adjust spacing between legend markers and labels
)

plt.tight_layout()
plt.show()
